In [1]:
import json

In [2]:
with open("dataset.json", "r") as f:
    datasets = json.load(f)

In [3]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset
import torch

In [4]:
model_name = "rinna/japanese-gpt2-medium"

#### モデルの選択

rinna/japanese-gpt2-medium
にしたのは、
rinna/japanese-gpt-1b
だと、GPUのスペック不足でtrain()が進行しなかったため。


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [7]:
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=7)
    labels = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=7)
    inputs['labels'] = labels['input_ids']
    return inputs

#### max_length

7じゃないとエラーになった

理由は不明・・・・

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs'
)

In [9]:
train_eval_border_index = round(len(datasets) * 0.8)

In [10]:
train_dataset = Dataset.from_list(datasets[:train_eval_border_index]).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_list(datasets[train_eval_border_index:]).map(tokenize_function, batched=True)

Map:   0%|          | 0/2460 [00:00<?, ? examples/s]

Map:   0%|          | 0/615 [00:00<?, ? examples/s]

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [12]:
trainer.train()

Step,Training Loss
500,2.539700
1000,1.516800
1500,1.313000


TrainOutput(global_step=1845, training_loss=1.6907642230108824, metrics={'train_runtime': 285.06, 'train_samples_per_second': 25.889, 'train_steps_per_second': 6.472, 'total_flos': 93704448983040.0, 'train_loss': 1.6907642230108824, 'epoch': 3.0})

In [13]:
prompt = "デオキシスのタイプを教えてください"

In [18]:
if torch.cuda.is_available():
    model = model.to("cuda")

input_ids = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=100, # 最長の文章長
        min_length=100, # 最短の文章長
        do_sample=True,
        top_k=500, # 上位{top_k}個の文章を保持
        top_p=0.95, # 上位{top_p}%の単語から選択する。例）上位95%の単語から選んでくる
        pad_token_id=tokenizer.pad_token_id,
    )
decoded = tokenizer.batch_decode(output, skip_special_tokens=True)

print(decoded)

['デオキシスのタイプを教えてください|サイホーンのタイプは? エメラルドのタイプは? デオキシスのタイプは? ...のタイプは? もし、ご自身で確かめてみたいというのであれば、 デオキシスのタイプは?のタイプは?のタイプは、 このページの上のほうに、デオキシスのタイプをのせていますので、 見てみてください。 サイホーンのタイプは?のタイプは、']
